In [1]:
# https://runawayhorse001.github.io/LearningApacheSpark/clustering.html
from utilsom import * 

In [2]:
#input_data = spark.read.csv("Cardiotocography_02_v10.csv", header=True, inferSchema=True, mode="DROPMALFORMED", encoding='UTF-8')
#df = df.drop("_c0")
#input_data.show(n=5,truncate=False)
#http://archive.ics.uci.edu/ml/datasets/HEPMASS  
input_data = load_data("hdfs://localhost:9000/data/Cardiotocography_02_v10.csv")
print(input_data.count())
print(input_data)

1688
DataFrame[_c0: int, LB: double, AC: double, FM: double, UC: double, ASTV: double, MSTV: double, ALTV: double, MLTV: double, DL: double, DS: double, DP: double, Width: double, Min: double, Max: double, Nmax: double, Nzeros: double, Mode: double, Mean: double, Median: double, Variance: double, Tendency: double, id: double, c90: string]


In [3]:
# dellet label
df_input_data = input_data.drop("_c0","c90","id")
# staep 2
df_input_data = shuffling(df_input_data, 10)

# staep 3
input_features = assembler(df_input_data,df_input_data.columns[0:])

# staep 4
scaledData = scaler(input_features)

# SOM

In [63]:
#https://machinelearningnepal.com/2018/01/22/apache-spark-implementation-of-som-batch-algorithm/
# step 5 Som
from som.batch_som import SOM
from matplotlib import pyplot as plt
from matplotlib import patches as patches

mapsize = calculate_map_size(input_data)
print('Som: X = {} and Y = {}, {} neurones'.format(mapsize[0], mapsize[1],mapsize[0]*mapsize[1]))

#x =  mapsize[0] 
#y = mapsize[1]
som = SOM(10,5,21) # (x×y) neurons, (70 features)
som.train(scaledData, 10) # data , epochs  +10

   # for show All codebook
#som.net
   # dimention of codebook (x,y,features)
#print('dimention of codebook 3D:',som.net.shape)

#som.net.reshpae(x*y,features)
data = som.net
data = data.transpose(2,1,0).reshape(-1,data.shape[2])
#print('dimention of codebook 2D:',data.shape)

Som: X = 15 and Y = 14, 210 neurones
SOM training epoches 1
neighborhood radius  4.256699612603923
-------------------------------------
SOM training epoches 2
neighborhood radius  3.623898318388478
-------------------------------------
SOM training epoches 3
neighborhood radius  3.0851693136000486
-------------------------------------
SOM training epoches 4
neighborhood radius  2.626527804403768
-------------------------------------
SOM training epoches 5
neighborhood radius  2.23606797749979
-------------------------------------
SOM training epoches 6
neighborhood radius  1.9036539387158788
-------------------------------------
SOM training epoches 7
neighborhood radius  1.6206565966927624
-------------------------------------
SOM training epoches 8
neighborhood radius  1.3797296614612151
-------------------------------------
SOM training epoches 9
neighborhood radius  1.1746189430880192
-------------------------------------
SOM training epoches 10
neighborhood radius  1.0
----------

In [64]:
# prediction som
# pour avoir bmu, bmu, index 
pre = som.predict(scaledData)
pre.show(2)
print(pre.count())

+---+----+----+---+---+---+----+-----+----+----+-----+-----+------+-----+-----+----+------+--------+---+--------+-----+--------------------+---------+--------------------+
| AC|ALTV|ASTV| DL| DP| DS|  FM|   LB|MLTV|MSTV|  Max| Mean|Median|  Min| Mode|Nmax|Nzeros|Tendency| UC|Variance|Width|                 bmu|  bmu_idx|            features|
+---+----+----+---+---+---+----+-----+----+----+-----+-----+------+-----+-----+----+------+--------+---+--------+-----+--------------------+---------+--------------------+
|1.0| 6.0|54.0|0.0|0.0|0.0| 0.0|130.0| 8.7| 0.6|146.0|133.0| 134.0|116.0|133.0| 2.0|   0.0|     0.0|9.0|     1.0| 30.0|[0.83294447868712...|[2.0,0.0]|[0.8125,0.0384615...|
|2.0|26.0|57.0|0.0|0.0|0.0|10.0|130.0|14.3| 0.6|156.0|131.0| 133.0| 65.0|129.0| 7.0|   0.0|     1.0|4.0|    10.0| 91.0|[0.83374089614232...|[8.0,4.0]|[0.8125,0.0769230...|
+---+----+----+---+---+---+----+-----+----+----+-----+-----+------+-----+-----+----+------+--------+---+--------+-----+--------------------+

In [65]:
# to kmenas, select, bmu 
bmux = pre.select('bmu','bmu_idx','features')
print(bmux.show(5))

+--------------------+---------+--------------------+
|                 bmu|  bmu_idx|            features|
+--------------------+---------+--------------------+
|[0.83294447868712...|[2.0,0.0]|[0.8125,0.0384615...|
|[0.83374089614232...|[8.0,4.0]|[0.8125,0.0769230...|
|[0.78584788203453...|[0.0,2.0]|[0.6875,0.1153846...|
|[0.78584788203453...|[0.0,2.0]|[0.6875,0.1923076...|
|[0.88754497082603...|[9.0,1.0]|[0.85,0.115384615...|
+--------------------+---------+--------------------+
only showing top 5 rows

None


# K-means

In [66]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.

featureIndexer = VectorIndexer(inputCol="bmu", \
                               outputCol="indexedbmu",\
                               ).fit(bmux)

data = featureIndexer.transform(bmux)

In [67]:
data.show(5,True)

+--------------------+---------+--------------------+--------------------+
|                 bmu|  bmu_idx|            features|          indexedbmu|
+--------------------+---------+--------------------+--------------------+
|[0.83294447868712...|[2.0,0.0]|[0.8125,0.0384615...|[0.83294447868712...|
|[0.83374089614232...|[8.0,4.0]|[0.8125,0.0769230...|[0.83374089614232...|
|[0.78584788203453...|[0.0,2.0]|[0.6875,0.1153846...|[0.78584788203453...|
|[0.78584788203453...|[0.0,2.0]|[0.6875,0.1923076...|[0.78584788203453...|
|[0.88754497082603...|[9.0,1.0]|[0.85,0.115384615...|[0.88754497082603...|
+--------------------+---------+--------------------+--------------------+
only showing top 5 rows



In [68]:
from pyspark.ml.clustering import KMeans, KMeansModel

kmeans = KMeans() \
          .setK(2) \
          .setFeaturesCol("indexedbmu")\
          .setPredictionCol("cluster")

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, kmeans])

model = pipeline.fit(bmux)

cluster = model.transform(bmux)

In [69]:
cluster = model.transform(bmux)

In [70]:
cluster.show(5)

+--------------------+---------+--------------------+--------------------+-------+
|                 bmu|  bmu_idx|            features|          indexedbmu|cluster|
+--------------------+---------+--------------------+--------------------+-------+
|[0.83294447868712...|[2.0,0.0]|[0.8125,0.0384615...|[0.83294447868712...|      1|
|[0.83374089614232...|[8.0,4.0]|[0.8125,0.0769230...|[0.83374089614232...|      0|
|[0.78584788203453...|[0.0,2.0]|[0.6875,0.1153846...|[0.78584788203453...|      1|
|[0.78584788203453...|[0.0,2.0]|[0.6875,0.1923076...|[0.78584788203453...|      1|
|[0.88754497082603...|[9.0,1.0]|[0.85,0.115384615...|[0.88754497082603...|      0|
+--------------------+---------+--------------------+--------------------+-------+
only showing top 5 rows



In [71]:
y_pred = cluster.select('cluster')

In [72]:
import numpy as np
y_pred = np.array(list(cluster.select('cluster').toPandas()['cluster']))

In [73]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
# load data

df = pd.read_csv("Cardiotocography_02_v10.csv")
df['c90'] = df.c90.apply(lambda label: 1 if label == "'yes'" else 0)
del df['id']
del df['Unnamed: 0']
# df to values
df = df.values
# 
y_test= df[:,21]

In [74]:
# predict
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of SOM K-means = {:.6f} %".format(acc * 100))
# Matrix de confusion
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

Test Accuracy of SOM K-means = 53.613744 %
[[887 768]
 [ 15  18]]


              precision    recall  f1-score   support

         0.0       0.98      0.54      0.69      1655
         1.0       0.02      0.55      0.04        33

   micro avg       0.54      0.54      0.54      1688
   macro avg       0.50      0.54      0.37      1688
weighted avg       0.96      0.54      0.68      1688

